#### Install and Import

In [ ]:
! pip install -q patchify
! pip install -q segmentation-models

     |████████████████████████████████| 58 kB 3.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 1.3 MB/s 


In [ ]:
import json
import os
import pathlib
import warnings
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import patchify
import PIL
import tensorflow as tf
from matplotlib import animation
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tqdm import tqdm

from Manager import InputMode, Manager
from Augment import Augment
from DiceLoss import DiceLoss
from DiceMetric import DiceCoefficient
from UNet import UNet

# import segmentation_models as sm

# from segmentation_models.losses import DiceLoss
# from segmentation_models.metrics import IOUScore

# sm.set_framework('tf.keras')
os.environ['TF_CUDNN_DETERMINISTIC'] = 'false'
os.environ['TF_DETERMINISTIC_OPS'] = 'false'
os.environ['TF_DISABLE_SEGMENT_REDUCTION_OP_DETERMINISM_EXCEPTIONS'] = 'true'


Segmentation Models: using `keras` framework.


#### Constants

In [ ]:

EPOCHS = 50
WIDTH = 256
HEIGHT = 256
BATCH_SIZE = 16
NUM_CLASSES = 1
VALIDATION_SPLIT = 0.2
SEED = 31415

CSV_ID = 1
MONITOR_MODE = 'loss'
SAVE_BEST = True
EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 7

REDUCE_LR = True
REDUCE_LR_FACTOR = 0.2
REDUCE_LR_PATIENCE = 3

THRESHOLD = 0.3

def display_overlay(image, mask):
    plt.figure(figsize = (7,7))
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)

### Predict Test

In [ ]:
! unzip -q saved_models.zip

In [ ]:
manager = Manager(loads_model=True, base_path='/content', training_id=0)

Model loaded from model_0


In [ ]:
def rle_encode(img, threshold=0.5):
    img = img.T
    pixels = np.concatenate((np.array([0]), img.flatten(), np.array([0])))
    beginings = np.where( (pixels >= threshold) & (np.concatenate((np.array([2]), pixels[:-1])) < threshold) )[0]
    ends = np.where( (pixels >= threshold) & (np.concatenate((pixels[1:], np.array([2]))) < threshold) )[0]

    rle = ''
    for begin, end in zip(beginings, ends):
        rle += str(begin-1) + ' ' + str(end - begin + 1) + ' '
    return rle[:-1]

def get_correct_dimension(number, divisible):
    if number % divisible == 0:
        return number
    else:
        return number + divisible - (number % divisible)

def correct_image_size(img, size):
    if img.shape[0] != size:
        img = cv2.resize(img, (size, size))
    return img


In [ ]:
test_images_dir = 'dataset/test_images'
test_images = [name for name in os.listdir(test_images_dir)]
test_images = [i for i in test_images if '.tiff' in i]
test_images = sorted(test_images, key=lambda x: int(x.split('.')[0]))

In [ ]:
PATCH_SIZE = 1024
submission_df = None
submission_dict = {'id': [], 'rle': []}
for test_image_filename in tqdm(test_images):
    image_id = int(test_image_filename.split('.')[0])
    submission_dict['id'].append(image_id)
    test_image_file = test_images_dir + '/' + test_image_filename
    image = cv2.imread(test_image_file)
    ORIGINAL_SIZE = image.shape[0]
    image = correct_image_size(image, get_correct_dimension(image.shape[0], PATCH_SIZE))
    CORRECTED_SIZE = image.shape[0]

    image = tf.cast(image, tf.float32) / 255.0

    test_mask = np.zeros((CORRECTED_SIZE, CORRECTED_SIZE))

    for i in range(0, image.shape[0], PATCH_SIZE):
        for j in range(0, image.shape[1], PATCH_SIZE):
            smaller_image = image[i:i+PATCH_SIZE, j:j+PATCH_SIZE]
            smaller_image = cv2.resize(smaller_image.numpy(),  (HEIGHT, WIDTH))
            smaller_image = tf.expand_dims(smaller_image, 0) # Create a batch
            smaller_mask = manager.model.predict(smaller_image)

            smaller_mask = np.where(smaller_mask > THRESHOLD, 1, 0)
            
            # smaller_mask = tf.math.argmax(smaller_mask, axis=-1)
            smaller_mask = tf.cast(smaller_mask, tf.float32)

            # smaller_mask = tf.expand_dims(smaller_mask[0], 2)
            smaller_mask = cv2.resize(smaller_mask[0].numpy(), (PATCH_SIZE, PATCH_SIZE))

            test_mask[i:i+PATCH_SIZE, j:j+PATCH_SIZE] = smaller_mask

    test_mask = correct_image_size(test_mask, ORIGINAL_SIZE)
    # plt.imshow(test_mask)
    cv2.imwrite(str(image_id) +'_mask.png', test_mask * 255)

    rle = rle_encode(test_mask)
    submission_dict['rle'].append(rle)

submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('submission.csv', index=False)

100%|██████████| 3/3 [00:02<00:00,  1.22it/s]


In [ ]:
submission_df

,id,rle
0,10044,1525731 30 1528731 31 1531730 32 1534729 34 15...
1,10078,40777 2 42799 4 44823 3 910818 4 912840 7 9148...
2,27879,1441268 30 1444266 34 1447266 35 1450265 37 14...
